In [ ]:
pip install opencv-python

In [ ]:
pip install numpy 

In [ ]:
pip install matplotlib

In [ ]:
pip install scipy

In [ ]:
pip install scikit-image

In [1]:
import cv2
import numpy as np
from matplotlib import pyplot as plt
from scipy import ndimage
from skimage import measure, color, io

In [2]:
img1 = cv2.imread("SNAP-132445-0096.jpg")

In [3]:
img = cv2.cvtColor(img1,cv2.COLOR_BGR2GRAY)

In [4]:
pixels_to_um = 0.5 # 1 pixel = 500 nm

In [5]:
ret1, thresh = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY+cv2.THRESH_OTSU)

In [6]:
kernel = np.ones((3,3),np.uint8)
opening = cv2.morphologyEx(thresh,cv2.MORPH_OPEN,kernel, iterations = 2)

In [7]:
from skimage.segmentation import clear_border
opening = clear_border(opening)

In [8]:
sure_bg = cv2.dilate(opening,kernel,iterations=2)

In [9]:
dist_transform = cv2.distanceTransform(opening,cv2.DIST_L2,3)

In [10]:
ret2, sure_fg = cv2.threshold(dist_transform,0.2*dist_transform.max(),255,0)

In [11]:
sure_fg = np.uint8(sure_fg)

In [12]:
unknown = cv2.subtract(sure_bg,sure_fg)

In [13]:
ret3, markers = cv2.connectedComponents(sure_fg)

In [14]:
markers = markers+10

In [15]:
markers[unknown==255] = 0

In [26]:
markers = cv2.watershed(img1,markers)

In [27]:
img1[markers == -1] = [0,255,255]  

In [28]:
img2 = color.label2rgb(markers, bg_label=0)

In [ ]:
cv2.imshow('Overlay on original image', img1)
cv2.imshow('Colored Grains', img2)
cv2.waitKey(0)

In [ ]:
regions = measure.regionprops(markers, intensity_image=img)

In [ ]:
propList = ['Area',
            'equivalent_diameter', #Added... verify if it works
            'orientation', #Added, verify if it works. Angle btwn x-axis and major axis.
            'MajorAxisLength',
            'MinorAxisLength',
            'Perimeter',
            'MinIntensity',
            'MeanIntensity',
            'MaxIntensity']   

In [ ]:
output_file = open('image_measurements.csv', 'w')
output_file.write('Grain #' + "," + "," + ",".join(propList) + '\n')

In [ ]:
grain_number = 1
for region_props in regions:
    output_file.write(str(grain_number) + ',')

In [ ]:
for i,prop in enumerate(propList):
        if(prop == 'Area'): 
            to_print = region_props[prop]*pixels_to_um**2   #Convert pixel square to um square
        elif(prop == 'orientation'): 
            to_print = region_props[prop]*57.2958  #Convert to degrees from radians
        elif(prop.find('Intensity') < 0):          # Any prop without Intensity in its name
            to_print = region_props[prop]*pixels_to_um
        else: 
            to_print = region_props[prop]     #Reamining props, basically the ones with Intensity in its name
        output_file.write(',' + str(to_print))
        output_file.write('\n')
        grain_number += 1
output_file.close() 